Hello.

I am looking to Scrape the website below for the following information:

https://www.honestdoor.com/map

Information required for every data point for properties located in the City of Edmonton:
- transaction date
- Transaction price
- Permits
- Honestdoor Price
- Year Built

In [ ]:
import requests
from time import sleep
import pickle
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from openpyxl import load_workbook
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
path = r'C:\Users\38099\OneDrive - AFG\Documents\Real estate parsing.xlsx'

In [237]:
# try:
#     browser.find_element_by_xpath('//*[@id = "intercom-modal-container"]/iframe')
#     iframe = browser.find_element_by_xpath('//*[@id = "intercom-modal-container"]/iframe')
#     browser.switch_to_frame(iframe)
#     browser.find_element_by_xpath('//span[@class = "intercom-post-close intercom-199syus e1uu5mk82"]').click()
# except NoSuchElementException as e:
#     print('Iframe is closed')
# sleep(4)
# browser.find_element_by_xpath('//button[contains(@class, "Knob__Root-z18gzs-0 dgHvkt")]').click()
# sleep(3)
# browser.find_element_by_xpath('//div[contains(@class,  "Gym-w9ppoe-0 ciitnz")]/label[contains(@class ,"Checkboxes__StyledCheckbox-sc-32nloj-0 kENcUO ant-checkbox-wrapper")]/span[1]/input[contains(@value, "Commercial")]').click()
# sleep(3)
# browser.find_element_by_xpath('//button[contains(@class,  "ant-btn Buttons__ApplyButton-sc-1gz2w5o-1 iplbkA ant-btn-sm")]').click()
# sleep(3)
# browser.find_element_by_xpath('//button[contains(text(), "List")]').click()

In [250]:
def main_page():
    browser.get('https://www.honestdoor.com/map')
    sleep(7)
    try:
        browser.find_element_by_xpath('//*[@id = "intercom-modal-container"]/iframe')
        iframe = browser.find_element_by_xpath('//*[@id = "intercom-modal-container"]/iframe')
        browser.switch_to_frame(iframe)
        browser.find_element_by_xpath('//span[@class = "intercom-post-close intercom-199syus e1uu5mk82"]').click()
    except NoSuchElementException as e:
        print('Iframe is closed')
    sleep(6)
    browser.find_element_by_xpath('//section[@class = "split-right"]/button[contains(@class, "Knob__Root-z18gzs-0 dgHvkt")]').click()
    sleep(6)
    browser.find_element_by_xpath('//section[@class = "section-filter"]/button[contains(@class, "Knob__Root-z18gzs-0 dgHvkt")]').click()
    sleep(6)


In [ ]:
def main_page():
    browser.get('https://www.honestdoor.com/map')
    sleep(7)
    try:
        browser.find_element_by_xpath('//*[@id = "intercom-modal-container"]/iframe')
        iframe = browser.find_element_by_xpath('//*[@id = "intercom-modal-container"]/iframe')
        browser.switch_to_frame(iframe)
        browser.find_element_by_xpath('//span[@class = "intercom-post-close intercom-199syus e1uu5mk82"]').click()
    except NoSuchElementException as e:
        print('Iframe is closed')
    sleep(6)
    browser.find_element_by_xpath('//section[@class = "split-right"]/button[contains(@class, "Knob__Root-z18gzs-0 dgHvkt")]').click()
    sleep(6)
    browser.find_element_by_xpath('//section[@class = "section-filter"]/button[contains(@class, "Knob__Root-z18gzs-0 dgHvkt")]').click()
    sleep(6)


In [568]:
def information(a):
    try:
        year_built = 0
        land_size = ''
        taxes = ''
        zoning = ''
        href = ''
        #information_df = pd.DataFrame(columns = ('href', 'YEAR BUILT', 'LAND SIZE','TAXES', 'ZONING'))
        for i in browser.find_elements_by_xpath('//*[@class="ant-col ant-col-xs-24 ant-col-sm-12"]'):
            if i.find_element_by_xpath('./div/dt').text == 'YEAR BUILT':
                year_built = i.find_element_by_xpath('./div/dd').text
            elif i.find_element_by_xpath('./div/dt').text == 'LAND SIZE':
                land_size = i.find_element_by_xpath('./div/dd').text.split('\n')
            elif i.find_element_by_xpath('./div/dt').text == 'TAXES':
                taxes = i.find_element_by_xpath('./div/dd').text.split('\n')
            elif i.find_element_by_xpath('./div/dt').text == 'ZONING':
                zoning = i.find_element_by_xpath('./div/dd').text

        if  len(taxes) == 2: 
            information_df = pd.DataFrame({'href': a, 'YEAR BUILT' : year_built, 'LAND SIZE' : land_size,'TAXES' : taxes, 'ZONING' : zoning})
        else:
            information_df = pd.DataFrame({'href': a, 'YEAR BUILT' : year_built, 'LAND SIZE' : land_size[0],'TAXES' : taxes, 'ZONING' : zoning})
    except:
        print('No main info')
        information_df = pd.DataFrame(columns = ('href', 'YEAR BUILT', 'LAND SIZE','TAXES', 'ZONING')) 
    return information_df

In [571]:
def transactions(a):
    try:
        global tr_data
        tr_data = pd.DataFrame(columns = ('href', 'Date', 'Amount'))
        for t in browser.find_elements_by_xpath('//*[@class="TransactionHistory__Root-sc-4qo0gb-0 ZuhVz"]'):
            transactions  = t.find_element_by_xpath('./table/tbody').text.split('\n')
            for t in transactions:
                tr = t.split(' ')
                tr_date = ' '.join([tr[0], tr[1]])
                tr_val = tr[2]
                transaction_df = pd.DataFrame({'href' : [a], 'Date' : tr_date, 'Amount' : tr_val})
                tr_data = tr_data.append(transaction_df)
    except:
        print('No transactions data')
        transaction_df = pd.DataFrame(columns = ('href', 'Date', 'Amount'))

    return tr_data

In [241]:
def assesments(a):
    try:
        sleep(3)
        assesments_arr = browser.find_element_by_xpath('//*[@class="PriceHistoryChartstyles__Root-m3dir-0 hUxtyk"]/div/div/div')
        assesments_price = assesments_arr.text.split('\n') 
        as_lngth = len(assesments_price)
        assesments_arr = browser.find_element_by_xpath('//*[@class="PriceHistoryChartstyles__Root-m3dir-0 hUxtyk"]/div/div').text.split('\n') 
        assesments_year = assesments_arr[:as_lngth]
        assesments_df = pd.DataFrame({'href': a, 'YEARS' : assesments_year, 'PRICE' : assesments_price})
    except NoSuchElementException as e:
        print('No assessmnts data')
        assesments_df = pd.DataFrame(columns = ('href', 'YEARS', 'PRICE'))
    return assesments_df

In [242]:
def neighbourhood(a):
    neighbourhood_arr = ''
    neighbourhood_year = ''
    try:
        browser.find_element_by_xpath('//li[contains(text(), "Neighbourhood Stats")]').click()
        sleep(3)
        neighbourhood_year= browser.find_element_by_xpath('//*[@class="NeighbourhoodAssessmentGraph__Root-t9qgdf-0 dfmGTl"]/div/div/*[name()="svg"]/*[name()="g"]').text.split('\n')
        neighbourhood_arr= browser.find_element_by_xpath('//*[@class="NeighbourhoodAssessmentGraph__Root-t9qgdf-0 dfmGTl"]/div/div/div')
        neighbourhood_arr = neighbourhood_arr.text.split('\n')

        if len(neighbourhood_arr) > 1:
            if len(neighbourhood_arr) >= len(neighbourhood_year)*2:
                n_lngth = int(len(neighbourhood_arr)/2)
                neighbourhood_house_price = neighbourhood_arr[:n_lngth]
                neighbourhood_condo_price = neighbourhood_arr[n_lngth:]
                neighbourhood_df = pd.DataFrame({'href': a, 'YEARS' : neighbourhood_year, 'HOUSE' : neighbourhood_house_price,'CONDO' : neighbourhood_condo_price})
            else:
                legend = browser.find_element_by_xpath('//*[@class="legend"]').text
                if legend == 'Condo':
                    neighbourhood_condo_price = neighbourhood_arr
                    neighbourhood_house_price = [0] * len(neighbourhood_arr)
                else:
                    neighbourhood_house_price = neighbourhood_arr
                    neighbourhood_condo_price = [0] * len(neighbourhood_arr)
                neighbourhood_df = pd.DataFrame({'href': a, 'YEARS' : neighbourhood_year, 'HOUSE' : neighbourhood_house_price,'CONDO' : neighbourhood_condo_price})
        else:
            print('else')
            neighbourhood_year = ''
            neighbourhood_house_price = ''
            neighbourhood_condo_price = ''
            neighbourhood_df = pd.DataFrame({'href': [a], 'YEARS' : [neighbourhood_year], 'HOUSE' : [neighbourhood_house_price],'CONDO' : [neighbourhood_condo_price]})

    except NoSuchElementException as e:
        print(e)
        neighbourhood_df = pd.DataFrame(columns = ('href', 'YEARS', 'HOUSE' ,'CONDO'))
        
    return neighbourhood_df

In [515]:
def permits(a):
    sleep(3)
    val_1 = []
    val_2 = []

    for i in browser.find_elements_by_css_selector('div.ant-col.ant-col-3'):
        val_1.append(i.text)
    # print(val_1)

    for i in browser.find_elements_by_css_selector('div.ant-col.ant-col-5'):
        val_2.append(i.text)
        
    if len(val_1) < 5:
        permits_df = pd.DataFrame(columns = ('href', 'APPLIED DATE','ISSUED DATE','JOB CATEGORY' ,'WORK TYPE' ,'EST. COST', 'JOB DESCRIPTION'))
    else :
        val_index = 1
        ins_i = 11
        x = []
        val_1.insert(5, 'JOB DESCRIPTION')
        x = [val_1[6:][i:i + 6] for i in range(0, len(val_1[6:]), 5)] 
        for r in x:
            try:
                r[5] = val_2[val_index]
                val_index += 1
            except:
                r.insert(5,val_2[val_index])
                val_index += 1
        for i in x : i.insert(0,a)
        permits_df = pd.DataFrame(x, columns = ('href', 'APPLIED DATE','ISSUED DATE','JOB CATEGORY' ,'WORK TYPE' ,'EST. COST', 'JOB DESCRIPTION'))

    return permits_df

In [570]:
information_data = pd.DataFrame(columns = ('href', 'YEAR BUILT', 'LAND SIZE','TAXES', 'ZONING'))
transaction_data = pd.DataFrame(columns = ('href', 'Date', 'Amount'))
assessments_data = pd.DataFrame(columns = ('href', 'YEARS', 'PRICE'))
neighbourhood_data = pd.DataFrame(columns = ('href', 'YEARS', 'HOUSE' ,'CONDO'))
permits_data = pd.DataFrame(columns = ('href', 'APPLIED DATE','ISSUED DATE','JOB CATEGORY' ,'WORK TYPE' ,'EST. COST', 'JOB DESCRIPTION'))

browser = webdriver.Chrome(r'C:\Users\38099\Downloads\chromedriver_win32_(1)\chromedriver.exe')
browser.maximize_window()
main_page()

for i in range(2,16):
    soup = BeautifulSoup(browser.page_source)
    hrefs = soup.find_all('a', {'class' : 'Anchor-sc-1pw8p89-0 bTOlMB'})
    for h in hrefs:
        link = 'https://www.honestdoor.com' + h['href']
        print(link)
        browser.get(link)
        inf_link_res = information(link)
        browser.find_element_by_xpath('//li[contains(text(), "Transactions")]').click()
        sleep(3)
        tr_link_res = transactions(link)
        browser.find_element_by_xpath('//li[contains(text(), "Assessments")]').click()
        as_link_res = assesments(link)
        browser.find_element_by_xpath('//li[contains(text(), "Neighbourhood Stats")]').click()
        ng_link_res = neighbourhood(link)
        browser.find_element_by_xpath('//li[contains(text(), "Permits")]').click()
        pm_link_res = permits(link)
        information_data = information_data.append(inf_link_res, ignore_index=True)    
        transaction_data = transaction_data.append(tr_link_res, ignore_index=True)
        assessments_data = assessments_data.append(as_link_res, ignore_index=True)
        neighbourhood_data = neighbourhood_data.append(ng_link_res, ignore_index=True)
        permits_data = permits_data.append(pm_link_res, ignore_index = True)
    
        # ОТСЛЕДИ ОШИБКУ !!!!!!!!!!!!!!!!!!!!!!!!!!1---------------------------!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
    try:
        main_page()
    except ElementClickInterceptedException as e:
        sleep(2)
        main_page()
        
    browser.execute_script("document.body.style.zoom = '100%'")
    browser.implicitly_wait(30)
    element = browser.find_element_by_xpath('//i[@class = "anticon anticon-right"]')
    sleep(5)
    browser.execute_script("arguments[0].scrollIntoView();", element)
    sleep(2)
    for num in range(1,i):
        element.click()
    sleep(5)
    
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

information_data.to_excel(writer, sheet_name = 'Information')
transaction_data.to_excel(writer, sheet_name = 'Transactions')
assessments_data.to_excel(writer, sheet_name = 'Assessments')
neighbourhood_data.to_excel(writer, sheet_name = 'Neighbours data')
permits_data.to_excel(writer, sheet_name = 'Permits')
writer.save()

c:\users\38099\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use driver.switch_to.frame instead
  import sys


https://www.honestdoor.com/property/308-4-avenue-sw-calgary-ab
['Oct 2019 $169.7M', 'Nov 2006 Unknown']
https://www.honestdoor.com/property/10155-102-street-nw-edmonton-ab
['Oct 2019 $126.5M', 'Jul 2000 Unknown']
https://www.honestdoor.com/property/504-6101-eton-blvd-sherwood-park-ab
No main info
['Jan 2020 $120M']
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class="NeighbourhoodAssessmentGraph__Root-t9qgdf-0 dfmGTl"]/div/div/*[name()="svg"]/*[name()="g"]"}
  (Session info: chrome=84.0.4147.89)

https://www.honestdoor.com/property/3699-63-avenue-ne-calgary-ab
['Nov 2019 $100.5M', 'Dec 2008 $102.65M']
https://www.honestdoor.com/property/10803-jasper-avenue-nw-edmonton-ab
['Jan 2020 $100M', 'Oct 2014 $10.4M']
https://www.honestdoor.com/property/179-skyview-circle-ne-calgary-ab-h98
['Dec 2019 $81.98M', 'Apr 2016 $6.09M']
https://www.honestdoor.com/property/6075-86-avenue-se-calgary-ab
['Jan 2019 $79.55M', 'Nov 2004 $6.42M']
else
https://www.honest

https://www.honestdoor.com/property/33-castleridge-boulevard-ne-calgary-ab
['Oct 2019 $17.47M', 'Nov 2013 $16.9M']
https://www.honestdoor.com/property/7612-17-street-nw-edmonton-ab
['Nov 2019 $17.1M', 'Jul 2010 $14.8M', 'Jul 2008 $14.8M', 'Jul 2006 $8M']
Iframe is closed
https://www.honestdoor.com/property/4320-80-avenue-se-calgary-ab
['Jan 2020 $17.04M', 'Jan 1997 Unknown']
else
https://www.honestdoor.com/property/6202-106-avenue-se-calgary-ab
['Jul 2019 $16.82M']
else
https://www.honestdoor.com/property/16703-82-street-nw-edmonton-ab-vzw
['Sep 2019 $16.1M']
https://www.honestdoor.com/property/10501-barlow-trail-se-calgary-ab
['Nov 2019 $15.2M', 'Oct 2012 $15.75M']
https://www.honestdoor.com/property/1122-16-avenue-sw-calgary-ab
['Jun 2020 $14.87M', 'Jan 2013 $11.42M']
https://www.honestdoor.com/property/3215-dunmore-road-se-medicine-hat-ab
No main info
['Jan 2019 $14.5M']
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class="NeighbourhoodAssess

else
https://www.honestdoor.com/property/27-v-325-manning-road-ne-calgary-ab
['Mar 2019 $11.9M', 'Nov 2008 Unknown']
else
https://www.honestdoor.com/property/31-v-325-manning-road-ne-calgary-ab
['Mar 2019 $11.9M', 'Nov 2008 Unknown']
else
https://www.honestdoor.com/property/39-v-325-manning-road-ne-calgary-ab
['Mar 2019 $11.9M', 'Nov 2008 Unknown']
else
https://www.honestdoor.com/property/44-v-325-manning-road-ne-calgary-ab
['Mar 2019 $11.9M', 'Nov 2008 Unknown']
else
https://www.honestdoor.com/property/45-v-325-manning-road-ne-calgary-ab
['Mar 2019 $11.9M', 'Nov 2008 Unknown']
else
Iframe is closed
https://www.honestdoor.com/property/33-v-325-manning-road-ne-calgary-ab
['Mar 2019 $11.9M', 'Nov 2008 Unknown']
else
https://www.honestdoor.com/property/46-v-325-manning-road-ne-calgary-ab
['Mar 2019 $11.9M', 'Nov 2008 Unknown']
else
https://www.honestdoor.com/property/47-v-325-manning-road-ne-calgary-ab
['Mar 2019 $11.9M', 'Nov 2008 Unknown']
else
https://www.honestdoor.com/property/48-v-3